In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs

from dask_gateway import Gateway
from dask.distributed import Client

gateway = Gateway()
cluster = gateway.new_cluster()
cluster.scale(10)
client = Client(cluster,timeout="120s")
cluster


In [18]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

model_name='"HadGEM3-GC31-LL"'
experiment='"ssp585"'

test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

theta = df.query('source_id =='+ model_name +' & table_id == "Omon" & variable_id == "thetao" & experiment_id =='+ experiment+'& grid_label == "gn" & member_id == "r1i1p1f3"')

uo = df.query('source_id =='+ model_name +' & table_id == "Omon" & variable_id == "uo" & experiment_id =='+ experiment+' & member_id == "r1i1p1f3"')
vo = df.query('source_id =='+ model_name +' & table_id == "Omon" & variable_id == "vo" & experiment_id =='+ experiment+' & member_id == "r1i1p1f3"')
wo = df.query('source_id =='+ model_name +' & table_id == "Omon" & variable_id == "wo" & experiment_id =='+ experiment+' & member_id == "r1i1p1f3"')
tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f3"')
tauv=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f3"')

hfls=df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "hfls" & experiment_id =='+ experiment+'& member_id == "r1i1p1f3"')
hfss=df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "hfss" & experiment_id =='+ experiment+'& member_id == "r1i1p1f3"')
rlds=df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "rlds" & experiment_id =='+ experiment+'& member_id == "r1i1p1f3"')
rlus=df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "rlus" & experiment_id =='+ experiment+'& member_id == "r1i1p1f3"')
rsds=df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "rsds" & experiment_id =='+ experiment+'& member_id == "r1i1p1f3"')
rsus=df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "rsus" & experiment_id =='+ experiment+'& member_id == "r1i1p1f3"')


def get_df(invar):
# this only needs to be created once
    gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
    zstore = invar.zstore.values[-1]

# create a mutable-mapping-style interface to the store
    mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
    outvar = xr.open_zarr(mapper, consolidated=True)

    return outvar

theta_pi=get_df(theta)
uo_pi=get_df(uo)
vo_pi=get_df(vo)
wo_pi=get_df(wo)
tauu_pi=get_df(tauu)
tauv_pi=get_df(tauv)
hfls_pi=get_df(hfls)
hfss_pi=get_df(hfss)
rlds_pi=get_df(rlds)
rlus_pi=get_df(rlus)
rsds_pi=get_df(rsds)
rsus_pi=get_df(rsus)


In [19]:
theta_eq=theta_pi.sel(j=slice(170,200),lev=slice(0,500)).isel(time=slice(0,200*12))

uo_eq=uo_pi.sel(j=slice(170,200)).sel(lev=slice(0,500)).isel(time=slice(0,200*12))
vo_eq=vo_pi.sel(j=slice(170,200)).sel(lev=slice(0,500)).isel(time=slice(0,200*12))
vo_eq=vo_eq.drop('vertices_latitude')
vo_eq=vo_eq.drop('vertices_longitude')
wo_eq=wo_pi.sel(j=slice(170,200)).sel(lev=slice(0,500)).isel(time=slice(0,200*12))
wo_eq=wo_eq.drop('vertices_latitude')
wo_eq=wo_eq.drop('vertices_longitude')
tauu_eq=tauu_pi.sel(lat=slice(-5,5)).isel(time=slice(0,200*12))

Q_pi=(rsds_pi['rsds']-rsus_pi['rsus'])-(rlus_pi['rlus']-rlds_pi['rlds'])-hfls_pi['hfls']-hfss_pi['hfss']

Q_eq=Q_pi.sel(lat=slice(-5,5)).isel(time=slice(0,200*12))
#theta_pi.nav_lat.load()

In [7]:
theta

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version


In [20]:
import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

#xesmf doesn't like it when I define the grid from the dataset, so I like to explicitly define input and output grids
# For a curvelinear grid, we need both lat/lon and lat/lon bounds

lonO=theta_eq.longitude
latO=theta_eq.latitude
#lonO_b=theta_eq.x_bnds
#latO_b=theta_eq.y_bnds


grid_in = {'lon': lonO, 'lat': latO} 

grid_out = xr.Dataset({'lat': (['lat'], np.arange(-5, 5, 1.0)),
                     'lon': (['lon'], np.arange(0, 359, 1)),})


regridder = xe.Regridder(grid_in, grid_out, 'bilinear')


thetao_regrid = regridder(theta_eq)
#thetao_deep_regrid = regridder(theta_pi_deep)
uo_regrid = regridder(uo_eq) 
vo_regrid = regridder(vo_eq)  
wo_regrid = regridder(wo_eq)  

using dimensions ('j', 'i') from data variable thetao as the horizontal dimensions for this dataset.
using dimensions ('j', 'i') from data variable uo as the horizontal dimensions for this dataset.
using dimensions ('j', 'i') from data variable vo as the horizontal dimensions for this dataset.
using dimensions ('j', 'i') from data variable wo as the horizontal dimensions for this dataset.


/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:473: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:473: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:473: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:473: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply

In [ ]:
#theta_eq.to_netcdf('theta_eq_'+'IPSL-CM6A-LR'+'_ens1.nc')
#theta_eq.load()
#theta_eq
theta_rg_eq=thetao_regrid.sel(lat=slice(-5,5)).mean('lat')
theta_rg_eq.load()
theta_rg_eq.to_netcdf('theta_eq_'+'HadGEM3-GC31-LL'+'_control.nc')

In [22]:
uo_rg_eq=uo_regrid.sel(lat=slice(-5,5)).mean('lat')
uo_rg_eq.load()
uo_rg_eq.to_netcdf('uo_eq_'+'HadGEM3-GC31-LL'+'_ens1.nc')

vo_rg_eq=abs(vo_regrid.sel(lat=slice(-5,5))*vo_regrid).lat.sel(lat=slice(-5,5))
vo_rg_eq=vo_rg_eq.mean('lat')
vo_rg_eq.load()
vo_rg_eq.to_netcdf('vo_eq_'+'HadGEM3-GC31-LL'+'_ens1.nc')

wo_rg_eq=wo_regrid.sel(lat=slice(-5,5)).mean('lat')
wo_rg_eq.load()
wo_rg_eq.to_netcdf('wo_eq_'+'HadGEM3-GC31-LL'+'_ens1.nc')

distributed.client - ERROR - Failed to reconnect to scheduler after 120.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/tornado/iostream.py", line 1391, in _do_ssl_handshake
    self.socket.do_handshake()
  File "/srv/conda/envs/notebook/lib/python3.8/ssl.py", line 1309, in do_handshake
    self._sslobj.do_handshake()
ssl.SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:1124)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/asyncio/events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "/srv/conda/envs/notebook/lib/

In [13]:
theta

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
207071,CMIP,MOHC,HadGEM3-GC31-LL,abrupt-4xCO2,r1i1p1f3,Omon,thetao,gn,gs://cmip6/CMIP6/CMIP/MOHC/HadGEM3-GC31-LL/abr...,NaN,20190620


In [13]:
tauu_eq=tauu_eq.mean('lat')
tauu_eq.load()
tauu_eq.to_netcdf('tauu_eq_'+'HadGEM3-GC31-LL'+'_4x.nc')

Q_eq=Q_eq.mean('lat')
Q_eq.load()
Q_eq.to_netcdf('Q_eq_'+'HadGEM3-GC31-LL'+'_4x.nc')

In [59]:
theta

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
396417,ScenarioMIP,MOHC,HadGEM3-GC31-LL,ssp585,r1i1p1f3,Omon,thetao,gn,gs://cmip6/CMIP6/ScenarioMIP/MOHC/HadGEM3-GC31...,NaN,20200114
